# 获取语料库

In [18]:
import pickle
import jieba

# 读取tfidf结果
with open(r'E:\obsidian\Master\fund_stream_project\codes\tfidf_result.pickle', 'rb') as f:

    pickle_result=pickle.load(f)
corpus=pickle_result[0]
txt_files=pickle_result[1]
X=pickle_result[2]
features=pickle_result[3]

# 从本地txt文件中读取筛选后的关键词
key_words_file = r'E:\obsidian\Master\fund_stream_project\codes\analyse\关键词.txt'
key_words = set()
with open(key_words_file, 'r', encoding='utf-8') as f:
    for line in f:
        key_words.add(line.strip())

In [4]:
txt_files[0].split('\\')[-1].split('.')[0]

'3850341'

In [19]:
from collections import defaultdict
wanted_sen = {}
for text_num in range(0,len(corpus)):
    text=corpus[text_num]
    text=text[text.find("文字记录:") + len("文字记录:"):].replace('\n','')
    try:
        code=int(txt_files[text_num].split('\\')[-1].split('.')[0])
    except:
        continue
    file_vector = X[text_num]
    sorted_indices = file_vector.toarray().argsort()[0, ::-1][:30]
    # 拿到文档前30个关键词
    key_list_30=[]
    for idx in sorted_indices:
        if features[idx] in key_words:
            key_list_30.append(features[idx])


    # 将文档拆成句子的列表
    sentences_list = []
    sentence = ''
    for word in text:
        if word in ['。', '！', '？']:
            sentence += word
            sentences_list.append(sentence)
            sentence = ''
        else:
            sentence += word

    # 将句子拆成词语的列表
    sent_doc=[]
    for sentence in sentences_list:
        words = list(jieba.cut(sentence, cut_all=False))
        sent_doc.append(words)
    
    
    has_same=[]
    for i in range(0,len(sent_doc)):
        has_same.append(set(sent_doc[i]).intersection(set(key_list_30))!=set()) # 交集非空表明有重复
    
    for i in range(0,len(sent_doc)):
        if has_same[i]:#本身含
            if str(code) not in wanted_sen:
                wanted_sen[str(code)]=[]
            else:
                wanted_sen[str(code)].append(sentences_list[i])
        elif i>1 and i<len(sent_doc)-2:  # 本身不含，不在首尾，看上下含不含
            if (has_same[i-2] or has_same[i-1]) and (has_same[i+2] or has_same[i+1]): #上下为2的窗口中有重复，则也加入
                if str(code) not in wanted_sen:
                    wanted_sen[str(code)]=[]
                else:
                    wanted_sen[str(code)].append(sentences_list[i])

with open('code_opinions_dict.pickle','wb') as f:
    pickle.dump(wanted_sen,f)

In [20]:
len(wanted_sen)

5568

# 注意
补充了新的txt之后要重跑一遍tfidf和上面的wanted——sen，生成每个文档的tfidf

## 使用腾讯词向量

In [21]:
from gensim.models import KeyedVectors

# 加载词向量模型
word_vectors = KeyedVectors.load_word2vec_format(r'E:\obsidian\Master\fund_stream_project\codes\predict\tencent-ailab-embedding-zh-d200-v0.2.0.txt', binary=False)

# 从词向量模型中获取单词向量
word = '苹果'  # 要向量化的单词
if word in word_vectors:
    vector = word_vectors[word]
    print(vector)
else:
    pass


[-0.461921 -0.014233 -0.048372  0.28874   0.306133  0.140307  0.539642
 -0.07467  -0.295719 -0.528008 -0.216229  0.569027 -0.073636  0.019376
  0.158029  0.385958 -0.619943  0.19712  -0.193381 -0.004257  0.030018
 -0.218658  0.095699  0.216874 -0.106251 -0.174064 -0.126379  0.119774
 -0.069757 -0.374367 -0.099812 -0.081158 -0.32928  -0.535561  0.793846
  0.109064  0.228561  0.343933 -0.262039 -0.222045 -0.395764 -0.107419
  0.082235 -0.283439 -0.121483  0.036244 -0.072383  0.344664  0.10446
  0.219135  0.327704 -0.12346   0.256317  0.004865  0.212775  0.156132
 -0.026614 -0.067437  0.61902   0.293585  0.331619  0.123144  0.290181
 -0.234849  0.283965  0.174604  0.374986 -0.247524 -0.016268 -0.059759
 -0.185743  0.073678  0.372429  0.499861 -0.052627 -0.11998  -0.218224
  0.139372 -0.157863 -0.363996  0.055498 -0.181125  0.020008 -0.140524
 -0.379319 -0.289876 -0.180254  0.029088 -0.029037 -0.063784  0.216772
  0.143372  0.358695  0.359362  0.223513  0.130438  0.218223 -0.208394
 -0.063

In [31]:
import jieba
from gensim.models import KeyedVectors
import numpy as np



wanted_sen_vec = {}



# 循环遍历 wanted_sen 字典中的每个键值对
for key in wanted_sen.keys():
    wanted_sen_vec[key] = []
    for sentence in wanted_sen[key]:
        # 分词
        sentence_cut = list(jieba.cut(sentence, cut_all=False))
        
        # 初始化句子向量
        sentence_vector = np.zeros((word_vectors.vector_size,), dtype="float32")
        
        # 遍历分词后的词语
        for word in sentence_cut:
            # 如果词在词向量模型中
            if word in word_vectors:
                # 将词向量加到句子向量上
                sentence_vector += word_vectors[word]
        
        # 简单平均得到句子向量
        sentence_vector /= len(sentence_cut)
        
        # 将句子向量添加到 wanted_sen_vec[key] 中
        wanted_sen_vec[key].append(sentence_vector)


with open('code_opinions_vec_dict_tencent.pickle','wb') as f:
    pickle.dump(wanted_sen_vec,f)


获得bert原始模型的向量字典  

In [3]:
from transformers import BertTokenizer, BertModel
from transformers import BertModel, BertConfig
import torch

# 下载并加载预训练的中文BERT模型和分词器
def load_pretrained_bert_model():
    # tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
    # model = BertModel.from_pretrained("bert-base-chinese")


    # 指定模型权重文件路径和配置文件路径
    model_weights_path = r"E:\obsidian\Master\fund_stream_project\codes\bert\model\pytorch_model.bin"
    model_config_path = r"E:\obsidian\Master\fund_stream_project\codes\bert\model\config.json"
        # 指定词汇表文件路径
    vocab_file_path = r"E:\obsidian\Master\fund_stream_project\codes\bert\model\vocab.txt"

    # 使用词汇表文件路径加载分词器
    tokenizer = BertTokenizer.from_pretrained(vocab_file_path)

    # 加载BERT配置
    config = BertConfig.from_pretrained(model_config_path)

    # 使用本地路径加载BERT模型
    model = BertModel.from_pretrained(model_weights_path, config=config)
    return tokenizer, model

# 加载预训练的BERT模型
tokenizer, model = load_pretrained_bert_model()

c:\Users\LJH\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LJH\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1645: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at E:\obsidian\Master\fund_stream_project\codes\bert\model\pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.de

In [4]:
# 对句子进行编码
def encode_sentence(sentence, tokenizer, model):
    # 使用分词器将句子分词并添加特殊标记
    inputs = tokenizer(sentence, return_tensors="pt", max_length=512, truncation=True)
    
    # 将输入传递给BERT模型并获取表示向量
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取句子的表示向量（CLS token的输出）
    sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1)
    
    return sentence_embedding[0].tolist()


# 对示例句子进行编码
# encoded_sentence = encode_sentence(example_sentence, tokenizer, model)



In [12]:
import pickle

with open('code_opinions_dict.pickle','rb') as f:
    wanted_sen=pickle.load(f)
wanted_sen_vec = {}


def process_line(line, max_length=450):
    if len(line) <= max_length:
        return [line]
    else:
        # 从450个字符的位置向前找到最近的“。”
        index = max_length
        while (index >= 0) and (line[index] != "。")and(line[index] != "?")and(line[index] != "？")and(line[index] != "！")and(line[index] != "!"):
            index -= 1
        
        # 如果没有找到句号，则找逗号
        if index == -1:
            index = max_length
            while (index >= 0) and (line[index] != ",")and (line[index] != "，"):
                index -= 1
        # 如果都没找到，则直接将字符串切割
        if index == -1:
            return [line[:max_length]] + process_line(line[max_length:], max_length)
        # 否则将句号之后的内容作为新行
        else:
            return [line[:index + 1]] + process_line(line[index + 1:], max_length)


# 循环遍历 wanted_sen 字典中的每个键值对
for key in wanted_sen.keys():
    wanted_sen_vec[key] = []
    for sentence in wanted_sen[key]:

        if len(sentence)>=450:  # bert的512个token限制
            sen_split=process_line(sentence)
            for sen in sen_split:
                wanted_sen_vec[key].append(encode_sentence(sen, tokenizer, model))

        else:
            wanted_sen_vec[key].append(encode_sentence(sentence, tokenizer, model))
        
        # 将句子向量添加到 wanted_sen_vec[key] 中



with open('code_opinions_vec_dict_bert_orig.pickle','wb') as f:
    pickle.dump(wanted_sen_vec,f)

KeyboardInterrupt: 

In [11]:
len(wanted_sen_vec['4285392'][0])

768

In [32]:
len(wanted_sen_vec)

5568

不管是用原始腾讯云向量、bert向量、bert增项训练向量，都得到一个字典，key是code，value是句子向量的列表。

# 预测模型代码一览

## 机器学习

In [3]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error



def svm(x_train, y_train,x_val, y_val):

    # 定义 SVM 模型
    svm = SVR()

    # 定义参数网格
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

    # 使用 GridSearchCV 寻找最佳参数
    grid_search = GridSearchCV(svm, param_grid, cv=3)  # 3 折交叉验证
    grid_search.fit(x_train, y_train)

    # 输出最佳参数
    print("Best parameters found: ", grid_search.best_params_)

    # 预测训练集和验证集
    y_pred_train = grid_search.predict(x_train)
    y_pred_val = grid_search.predict(x_val)

    # 计算训练集和验证集上的均方根误差
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    print("RMSE (Train): {:.5f}     RMSE (Val): {:.5f}".format(rmse_train,rmse_val))

def random_forest(x_train, y_train, x_val, y_val):

    # 定义随机森林模型
    rf = RandomForestRegressor()

    # 定义参数网格
    param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}

    # 使用 GridSearchCV 寻找最佳参数
    grid_search = GridSearchCV(rf, param_grid, cv=3)  # 3 折交叉验证
    grid_search.fit(x_train, y_train)

    # 输出最佳参数
    print("Best parameters found: ", grid_search.best_params_)

    # 预测训练集和验证集
    y_pred_train = grid_search.predict(x_train)
    y_pred_val = grid_search.predict(x_val)

    # 计算训练集和验证集上的均方根误差
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    print("RMSE (Train): {:.5f}     RMSE (Val): {:.5f}".format(rmse_train,rmse_val))



def xgboost_model(x_train, y_train, x_val, y_val):
    # 定义XGBoost模型
    xgb = XGBRegressor()

    # 定义参数网格
    param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}

    # 使用 GridSearchCV 寻找最佳参数
    grid_search = GridSearchCV(xgb, param_grid, cv=3)  # 3 折交叉验证
    grid_search.fit(x_train, y_train)

    # 输出最佳参数
    print("Best parameters found: ", grid_search.best_params_)

    # 预测训练集和验证集
    y_pred_train = grid_search.predict(x_train)
    y_pred_val = grid_search.predict(x_val)

    # 计算训练集和验证集上的均方根误差
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    # 打印训练集上的均方根误差（RMSE），限制小数位数为5位
    print("RMSE (Train): {:.5f}     RMSE (Val): {:.5f}".format(rmse_train,rmse_val))




## 深度学习

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def train_regression_model(x_train, y_train, x_val, y_val, input_size, output_size,
                            learning_rate=0.001, num_epochs=100, batch_size=16, weight_decay=1e-5,verbose=0):
    # 检查GPU是否可用
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 定义深度神经网络模型
    class SmallDNN(nn.Module):  
        def __init__(self, input_size, output_size):
            super(SmallDNN, self).__init__()
            self.fc1 = nn.Linear(input_size, 64)
            self.fc2 = nn.Linear(64, 32)
            self.fc3 = nn.Linear(32, output_size)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.5)  # 添加 dropout 层

        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)  # 在每一层后面添加 dropout
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.fc3(x)
            return x

    # 数据准备并移到GPU
    x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
    x_val = torch.tensor(x_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.float32).to(device)

    # 创建数据加载器
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # 初始化模型并移到GPU、损失函数和优化器
    model = SmallDNN(input_size, output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)  # 设置权重衰减参数

    # 训练模型
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            # 添加正则化项到损失函数
            l2_regularization = torch.tensor(0., device=device)
            for param in model.parameters():
                l2_regularization += torch.norm(param)
            loss += weight_decay * l2_regularization
            loss.backward()
            optimizer.step()
        if verbose==1:
            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 在训练集和验证集上进行预测
    with torch.no_grad():
        y_pred_train = model(x_train)
        y_pred_val = model(x_val)


    # 计算训练集和验证集上的均方根误差
    rmse_train = np.sqrt(mean_squared_error(y_train.cpu().numpy(), y_pred_train.cpu().numpy()))
    rmse_val = np.sqrt(mean_squared_error(y_val.cpu().numpy(), y_pred_val.cpu().numpy()))
    print("RMSE (Train): {:.5f}     RMSE (Val): {:.5f}".format(rmse_train,rmse_val))

    return model




# 首先获得金融指标数据，

In [37]:
import pandas as pd
import numpy as np
input_text=pd.read_csv(r"E:\obsidian\Master\fund_stream_project\codes\predict_input.csv")
input_fliter_txt_num=input_text[input_text['txt数量']>=2]
input_fliter_txt_num

,日期,txt数量,对数收益率,历史n3波动率,历史n5波动率,历史n10波动率,历史n20波动率,历史n40波动率,未来n3波动率,未来n5波动率,未来n10波动率,未来n20波动率,val
0,2024/2/22,14.0,0.0127,0.012594,0.055299,0.060961,0.211239,0.245277,0.197279,0.146276,0.225252,0.163822,1
1,2024/2/21,14.0,0.0097,0.009612,0.073098,0.064239,0.232260,0.255545,0.194860,0.144915,0.202274,0.169440,0
2,2024/2/20,11.0,0.0042,0.004179,0.076223,0.142469,0.253269,0.253710,0.193622,0.046082,0.130469,0.169322,0
3,2024/2/19,7.0,0.0156,0.015456,0.017868,0.213397,0.257556,0.253407,0.193750,0.055299,0.119561,0.169161,0
4,2024/2/8,6.0,0.0128,0.012712,0.137220,0.272480,0.244696,0.247520,0.189604,0.073098,0.067074,0.177217,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2023/6/6,24.0,-0.0115,-0.011544,0.127495,0.104702,0.099912,0.113817,0.128791,0.033721,0.038873,0.068473,1
173,2023/6/5,17.0,0.0007,0.000733,0.056534,0.070582,0.110569,0.113340,0.125928,0.111052,0.097647,0.088538,0
174,2023/6/2,17.0,0.0079,0.007907,0.091098,0.071846,0.113540,0.133386,0.126627,0.091963,0.097677,0.082532,0
175,2023/6/1,26.0,0.0000,0.000022,0.049330,0.054024,0.101923,0.129157,0.126330,0.127495,0.105232,0.085369,1


In [38]:

x_label=['对数收益率', '历史n3波动率', '历史n5波动率', '历史n10波动率', '历史n20波动率', '历史n40波动率']
y_label='未来n3波动率'

# 选择特定列
x_train = input_fliter_txt_num[input_fliter_txt_num['val'] == 0][x_label].to_numpy()
y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
x_val = input_fliter_txt_num[input_fliter_txt_num['val'] == 1][x_label].to_numpy()
y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()


## 不加特征，直接预测

In [6]:
# svm
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    svm(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.02853     RMSE (Val): 0.03055
------------未来n5波动率----------------
Best parameters found:  {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.05636     RMSE (Val): 0.05753
------------未来n10波动率----------------
Best parameters found:  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
RMSE (Train): 0.05460     RMSE (Val): 0.05595
------------未来n20波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
RMSE (Train): 0.06095     RMSE (Val): 0.06154


In [42]:
# 随机森林
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    random_forest(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'max_depth': 10, 'n_estimators': 100}
RMSE (Train): 0.00390     RMSE (Val): 0.01050
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 20, 'n_estimators': 300}
RMSE (Train): 0.02171     RMSE (Val): 0.05208
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 20, 'n_estimators': 200}
RMSE (Train): 0.01944     RMSE (Val): 0.04377
------------未来n20波动率----------------
Best parameters found:  {'max_depth': None, 'n_estimators': 300}
RMSE (Train): 0.01427     RMSE (Val): 0.02950


In [43]:
# xgboost
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    xgboost_model(x_train, y_train,x_val, y_val)


------------未来n3波动率----------------


Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.00122     RMSE (Val): 0.01475
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.00343     RMSE (Val): 0.05759
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 200}
RMSE (Train): 0.00135     RMSE (Val): 0.05250
------------未来n20波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 200}
RMSE (Train): 0.00144     RMSE (Val): 0.03488


In [42]:
# 深度
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    trained_model = train_regression_model(x_train, y_train, x_val, y_val, input_size=x_train.shape[1], output_size=1)

------------未来n3波动率----------------
RMSE (Train): 0.01979     RMSE (Val): 0.02142
------------未来n5波动率----------------
RMSE (Train): 0.01702     RMSE (Val): 0.02037
------------未来n10波动率----------------
RMSE (Train): 0.01872     RMSE (Val): 0.02021
------------未来n20波动率----------------
RMSE (Train): 0.01826     RMSE (Val): 0.01774


# 计算日向量

-  读取句向量

In [6]:
import pickle
# with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\code_opinions_vec_dict_tencent.pickle','rb') as f:
#     vec_dict=pickle.load(f)
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\code_opinions_vec_dict_bert_orig.pickle','rb') as f:
    vec_dict=pickle.load(f)
# with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\code_opinions_vec_dict_bert_fted.pickle','rb') as f:
#     vec_dict=pickle.load(f)



需要一个 日期-code  字典

In [3]:
import os
import pymysql
import requests
import pandas as pd
# 文件夹路径
folder_path = r"C:\Users\LJH\Desktop\txt存放\飞书"

# 获取文件夹内所有txt文件的路径
txt_files = [f.split('.')[0] for f in os.listdir(folder_path) if f.endswith('.txt')]
db = pymysql.connect(host='bj-cynosdbmysql-grp-igalwqqk.sql.tencentcdb.com',
                        user='root',
                        password='UIBE_chat_2023',
                        database='fund_stream',
                        charset='utf8mb4',
                        port=25445,)
cursor = db.cursor()
date_code_dict={}
for code in txt_files:
    query="select date from total where code={}".format(code)
    cursor.execute(query)
    result=cursor.fetchone()[0]
    if str(result) not in date_code_dict.keys():
        date_code_dict[str(result)]=[code]
    else:
        date_code_dict[str(result)].append(code)
date_code_dict

{'2023-06-13': ['4285392',
  '4303330',
  '4303690',
  '4303700',
  '4303713',
  '4303731',
  '4303797',
  '4303903',
  '4304336',
  '4304376',
  '4304434',
  '4304479',
  '4304481',
  '4304491',
  '4304498',
  '4304505',
  '4304514',
  '4304516',
  '4304593',
  '4304685',
  '4304722',
  '4304724',
  '4304738',
  '4304743',
  '4304748',
  '4304763',
  '4304773',
  '4304784',
  '4304793',
  '4304814',
  '4304869',
  '4304907',
  '4304909',
  '4304930',
  '4304972',
  '4305070',
  '4305270'],
 '2023-05-31': ['4294209',
  '4295537',
  '4297066',
  '4299052',
  '4299064',
  '4299534',
  '4299575',
  '4299586',
  '4299600',
  '4299851',
  '4299897'],
 '2023-06-02': ['4294796',
  '4298440',
  '4298827',
  '4298867',
  '4298982',
  '4299689',
  '4300266',
  '4300290',
  '4300296',
  '4300422',
  '4300577',
  '4300586',
  '4300594',
  '4300635',
  '4300640',
  '4300735',
  '4300985'],
 '2023-06-07': ['4295550',
  '4300891',
  '4301177',
  '4301256',
  '4301317',
  '4301378',
  '4301451',
  '43

In [4]:
import pickle
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\date_code_dict.pickle','wb')as f :
    pickle.dump(date_code_dict,f)

In [ ]:
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\date_code_dict.pickle','rb')as f :
    date_code_dict=pickle.load(f)

- 计算日内所有句向量所有维度的均值、方差、最小值、最大值、第一四分位数、第二四分位数、第三四分位数、偏度、峰度，并变成一个日向量

In [7]:
import numpy as np
from scipy.stats import skew, kurtosis
import pickle
count=0
dimen=768
day_vec={}
for date in input_fliter_txt_num['日期'].str.replace('/', '-'):
    date_m=date
    # 初始化一个空数组
    merged_array = np.empty((0, dimen))  
    try:
        code_list=date_code_dict[date] # 一天的所有txt的code
    except:
        mon=date.split('-')[1]
        if len(mon)==1:
            mon="0"+mon
        day=date.split('-')[2]
        if len(day)==1:
            day="0"+day
        year=date.split('-')[0]
        date_m='-'.join([year,mon,day])
        try:
            code_list=date_code_dict[date_m] # 一天的所有txt的code
        except:
            print("{}日期没找到txt".format(date_m))
            pass
        
    for code in code_list:
        # print(code)

        if code not in vec_dict.keys():
            print("{}not in vec_dict.keys()".format(code))
        else:
            if len(vec_dict[code])==0:
                continue
            merged_array = np.vstack((merged_array, vec_dict[code]))

    # 计算每个维度的统计量
    mean = np.mean(merged_array, axis=0)
    variance = np.var(merged_array, axis=0)
    min_val = np.min(merged_array, axis=0)
    max_val = np.max(merged_array, axis=0)
    quartiles = np.percentile(merged_array, [25, 50, 75], axis=0)
    quartile1, quartile2, quartile3 = quartiles
    skewness = skew(merged_array, axis=0)
    kurt = kurtosis(merged_array, axis=0)

    # 将统计量合并成一个向量
    stats_vector = np.concatenate((mean, variance, min_val, max_val, quartile1, quartile2, quartile3, skewness, kurt))
    day_vec[date_m]=stats_vector
    count+=1
    print(count)
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\day_vec_bert_orig.pickle','wb') as f:
    pickle.dump(day_vec,f)
# with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\day_vec_bert_fted.pickle','wb') as f:
#     pickle.dump(day_vec,f)
# with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\day_vec_tencent.pickle','wb') as f:
#     pickle.dump(day_vec,f)

1
4401430not in vec_dict.keys()
2
3
4
5
6
7
8
9
10
11
4395823not in vec_dict.keys()
12
13
14
15
16
17
18
19
20
21
22
23
24
4388887not in vec_dict.keys()
25
26
4387622not in vec_dict.keys()
27
28
29
4384874not in vec_dict.keys()
30
31
32
33
34
35
36
37
38
4381503not in vec_dict.keys()
39
4381342not in vec_dict.keys()
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
4364287not in vec_dict.keys()
66
67
68
69
4363474not in vec_dict.keys()
70
71
72
73
74
75
4354979not in vec_dict.keys()
76
77
78
4358840not in vec_dict.keys()
79
4357729not in vec_dict.keys()
80
81
82
83
84
85
86
4354255not in vec_dict.keys()
87
4353269not in vec_dict.keys()
88
89
4351122not in vec_dict.keys()
90
4349953not in vec_dict.keys()
91
92
93
94
95
96
4347370not in vec_dict.keys()
97
98
99
100
101
102
103
104
105
106
107
108
4338299not in vec_dict.keys()
109
4337678not in vec_dict.keys()
4337848not in vec_dict.keys()
110
111
112
4335961not in vec_dict.keys()
113
114
115
116
117
118
119
12

## 选择日向量加入X中

- 选择一个向量
---

In [7]:
import pickle
# 腾讯向量
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\day_vec_tencent.pickle','rb')as f :
    day_vec_dict=pickle.load(f)

In [13]:
import pickle
# 原始bert向量
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\day_vec_bert_orig.pickle','rb')as f :
    day_vec_dict=pickle.load(f)

In [23]:
import pickle
# 增强bert向量
with open(r'E:\obsidian\Master\fund_stream_project\codes\predict\day_vec_bert_fted.pickle','rb')as f :
    day_vec_dict=pickle.load(f)

---

如果是腾讯就len(day_vec_dict[date_m])!=1800，bert是9*768=6912
- 插入日向量

In [24]:
import pandas as pd
import numpy as np

def tran_date(date):
    date=date.replace('/','-')
    if len(date.split('-'))!=3:
        print(date)
    mon=date.split('-')[1]
    if len(mon)==1:
        mon="0"+mon
    day=date.split('-')[2]
    if len(day)==1:
        day="0"+day
    year=date.split('-')[0]
    date_m='-'.join([year,mon,day])

    return date_m

# 根据日期查找相应的向量
def find_vector(row):
    date = row['日期']
    if tran_date(date) in day_vec_dict:
        vector = day_vec_dict[tran_date(date)]
        vector_series = pd.Series(vector, index=[f'向量_{i+1}' for i in range(len(vector))])
        row = pd.concat([row, vector_series])  # 使用 pd.concat() 方法将两个 Series 连接起来
    return row

# 应用函数到每一行,把日向量接上去
result = input_fliter_txt_num.apply(find_vector, axis=1)

# 要排除的列名列表
excluded_columns = ['日期', 'txt数量', '未来n3波动率', '未来n5波动率', '未来n10波动率', '未来n20波动率']

# 使用列表理解构建一个新的列名列表，包括除了排除列以外的所有列
columns_to_keep = [col for col in result.columns if col not in excluded_columns]

# 使用这个列名列表来重新选择 DataFrame 中的列
df_filtered = result[columns_to_keep]

# 选择特定列
x_train = df_filtered[df_filtered['val'] == 0].drop('val', axis=1).to_numpy()
x_val = df_filtered[df_filtered['val'] == 1].drop('val', axis=1).to_numpy()

In [27]:
x_train.shape

(133, 6918)

## 腾讯向量预测

In [10]:
# svm
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    svm(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.02853     RMSE (Val): 0.03055
------------未来n5波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
RMSE (Train): 0.06274     RMSE (Val): 0.05823
------------未来n10波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.05850     RMSE (Val): 0.07916
------------未来n20波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
RMSE (Train): 0.06117     RMSE (Val): 0.06122


In [11]:
# random_forest

fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    random_forest(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'max_depth': 20, 'n_estimators': 200}
RMSE (Train): 0.00855     RMSE (Val): 0.02623
------------未来n5波动率----------------
Best parameters found:  {'max_depth': None, 'n_estimators': 200}
RMSE (Train): 0.02351     RMSE (Val): 0.05264
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 10, 'n_estimators': 200}
RMSE (Train): 0.02235     RMSE (Val): 0.04643
------------未来n20波动率----------------
Best parameters found:  {'max_depth': None, 'n_estimators': 200}
RMSE (Train): 0.01728     RMSE (Val): 0.04588


In [12]:
# xgboost_model
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    xgboost_model(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'max_depth': 5, 'n_estimators': 100}
RMSE (Train): 0.00055     RMSE (Val): 0.02844
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 5, 'n_estimators': 100}
RMSE (Train): 0.00066     RMSE (Val): 0.05799
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 5, 'n_estimators': 100}
RMSE (Train): 0.00061     RMSE (Val): 0.05102
------------未来n20波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 200}
RMSE (Train): 0.00072     RMSE (Val): 0.05248


In [157]:
# 深度
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    trained_model = train_regression_model(x_train, y_train, x_val, y_val, input_size=x_train.shape[1], output_size=1)

------------未来n3波动率----------------


RMSE (Train): 0.02155     RMSE (Val): 0.02232
------------未来n5波动率----------------
RMSE (Train): 0.02154     RMSE (Val): 0.02213
------------未来n10波动率----------------
RMSE (Train): 0.02134     RMSE (Val): 0.02212
------------未来n20波动率----------------
RMSE (Train): 0.02805     RMSE (Val): 0.02992


## 原始bert预测

In [16]:
# svm
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    svm(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.02853     RMSE (Val): 0.03055
------------未来n5波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.06306     RMSE (Val): 0.05546
------------未来n10波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.06567     RMSE (Val): 0.06122
------------未来n20波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.06335     RMSE (Val): 0.06347


In [17]:
# random_forest

fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    random_forest(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'max_depth': 20, 'n_estimators': 100}
RMSE (Train): 0.02131     RMSE (Val): 0.02241
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 10, 'n_estimators': 100}
RMSE (Train): 0.05613     RMSE (Val): 0.04987
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 20, 'n_estimators': 100}
RMSE (Train): 0.05442     RMSE (Val): 0.04826
------------未来n20波动率----------------
Best parameters found:  {'max_depth': None, 'n_estimators': 100}
RMSE (Train): 0.05043     RMSE (Val): 0.04433


In [18]:
# xgboost_model

fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    xgboost_model(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.02131     RMSE (Val): 0.02242
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.05613     RMSE (Val): 0.04986
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.05442     RMSE (Val): 0.04823
------------未来n20波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.05042     RMSE (Val): 0.04426


In [145]:
# 深度
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    trained_model = train_regression_model(x_train, y_train, x_val, y_val, input_size=x_train.shape[1], output_size=1)

------------未来n3波动率----------------


RMSE (Train): 0.02105     RMSE (Val): 0.02281
------------未来n5波动率----------------
RMSE (Train): 0.02396     RMSE (Val): 0.02289
------------未来n10波动率----------------
RMSE (Train): 0.02177     RMSE (Val): 0.02198
------------未来n20波动率----------------
RMSE (Train): 0.02121     RMSE (Val): 0.02182


## 增强bert

In [174]:
# svm
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    svm(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
RMSE (Train): 0.02853     RMSE (Val): 0.03055
------------未来n5波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
RMSE (Train): 0.06291     RMSE (Val): 0.05862
------------未来n10波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
RMSE (Train): 0.06745     RMSE (Val): 0.06621
------------未来n20波动率----------------
Best parameters found:  {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
RMSE (Train): 0.06153     RMSE (Val): 0.06245


In [122]:
# random_forest

fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    random_forest(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------


Best parameters found:  {'max_depth': 20, 'n_estimators': 200}
RMSE (Train): 0.02131     RMSE (Val): 0.02241
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 10, 'n_estimators': 300}
RMSE (Train): 0.05613     RMSE (Val): 0.04992
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 10, 'n_estimators': 100}
RMSE (Train): 0.05442     RMSE (Val): 0.04823
------------未来n20波动率----------------
Best parameters found:  {'max_depth': 20, 'n_estimators': 100}
RMSE (Train): 0.05043     RMSE (Val): 0.04422


In [22]:
# xgboost_model

fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    y_train=input_fliter_txt_num[input_fliter_txt_num['val'] == 0][y_label].to_numpy()
    y_val=input_fliter_txt_num[input_fliter_txt_num['val'] == 1][y_label].to_numpy()
    xgboost_model(x_train, y_train,x_val, y_val)

------------未来n3波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.02131     RMSE (Val): 0.02242
------------未来n5波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.05613     RMSE (Val): 0.04986
------------未来n10波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.05442     RMSE (Val): 0.04823
------------未来n20波动率----------------
Best parameters found:  {'max_depth': 3, 'n_estimators': 100}
RMSE (Train): 0.05042     RMSE (Val): 0.04426


In [29]:
# 深度
fore_day=['3','5','10','20']
for day in fore_day:
    y_label='未来n'+day+'波动率'
    print("------------"+y_label+"----------------")
    trained_model = train_regression_model(x_train, y_train, x_val, y_val, input_size=x_train.shape[1], output_size=1,verbose=0)

------------未来n3波动率----------------
RMSE (Train): 0.05064     RMSE (Val): 0.04419
------------未来n5波动率----------------
RMSE (Train): 0.05041     RMSE (Val): 0.04441
------------未来n10波动率----------------
RMSE (Train): 0.05045     RMSE (Val): 0.04405
------------未来n20波动率----------------
RMSE (Train): 0.05053     RMSE (Val): 0.04378


# 降维

## 自编码

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def autoencoder_dimensionality_reduction(x_train, x_val, hidden_size=None, learning_rate=0.001, num_epochs=200, batch_size=16):
    # 将数据转移到 GPU 上
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)

    # 合并训练数据和验证数据
    combined_data = torch.cat([x_train_tensor, x_val_tensor], dim=0)

    # 创建数据加载器
    train_loader = DataLoader(combined_data, batch_size=batch_size, shuffle=True)

    # 确定隐藏层大小（如果未指定）
    input_size = x_train.shape[1]
    if hidden_size is None:
        hidden_size = input_size // 2  # 默认隐藏层大小为输入特征的一半

    # 初始化自编码器模型
    model = Autoencoder(input_size, hidden_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练自编码器模型
    for epoch in range(num_epochs):
        for inputs in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)  # 自编码器的目标是使重构尽可能接近输入
            loss.backward()
            optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 在训练集和验证集上进行降维
    with torch.no_grad():
        encoded_x_train = model.encoder(x_train_tensor).cpu().numpy()
        encoded_x_val = model.encoder(x_val_tensor).cpu().numpy()

    return encoded_x_train, encoded_x_val
############################第一次
x_train_back=x_train
x_val_back=x_val
x_train, x_val= autoencoder_dimensionality_reduction(x_train_back, x_val_back, hidden_size=50)
###########################

##### 仅重新降维
# x_train, x_val= autoencoder_dimensionality_reduction(x_train_back, x_val_back, hidden_size=1000)

Epoch [10/200], Loss: 0.4631
Epoch [20/200], Loss: 0.4478
Epoch [30/200], Loss: 0.4341
Epoch [40/200], Loss: 0.4109
Epoch [50/200], Loss: 0.4114
Epoch [60/200], Loss: 0.4035
Epoch [70/200], Loss: 0.4040
Epoch [80/200], Loss: 0.3944
Epoch [90/200], Loss: 0.3811
Epoch [100/200], Loss: 0.3840
Epoch [110/200], Loss: 0.3731
Epoch [120/200], Loss: 0.3772
Epoch [130/200], Loss: 0.3668
Epoch [140/200], Loss: 0.3660
Epoch [150/200], Loss: 0.3604
Epoch [160/200], Loss: 0.3629
Epoch [170/200], Loss: 0.3636
Epoch [180/200], Loss: 0.3604
Epoch [190/200], Loss: 0.3609
Epoch [200/200], Loss: 0.3630


## PCA


In [97]:
from sklearn.decomposition import PCA

def pca_dimensionality_reduction(x_train, x_val, variance_threshold=0.95):
    # 创建 PCA 模型
    pca = PCA()

    # 拟合 PCA 模型并获取所有主成分的方差解释率
    pca.fit(x_train)

    # 根据方差解释率选择维度
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(cumulative_variance >= variance_threshold) + 1

    # 使用选定的维度重新拟合 PCA 模型并进行降维
    pca = PCA(n_components=n_components)
    x_train_reduced = pca.fit_transform(x_train)
    x_val_reduced = pca.transform(x_val)

    return x_train_reduced, x_val_reduced

# 使用示例
x_train, x_val = pca_dimensionality_reduction(x_train, x_val)


In [120]:
x_train.shape

(133, 50)

In [19]:
type(day_vec['2023-09-18'])

numpy.ndarray

In [80]:
day_vec.keys()

dict_keys(['2024-2-22', '2024-2-21', '2024-2-20', '2024-2-19', '2024-2-8', '2024-2-7', '2024-2-6', '2024-2-5', '2024-2-2', '2024-2-1', '2024-1-31', '2024-1-30', '2024-1-29', '2024-1-26', '2024-1-25', '2024-1-24', '2024-1-23', '2024-1-22', '2024-1-19', '2024-1-18', '2024-1-17', '2024-1-16', '2024-1-15', '2024-1-12', '2024-1-11', '2024-1-10', '2024-1-9', '2024-1-8', '2024-1-5', '2024-1-4', '2024-1-3', '2024-1-2', '2023-12-29', '2023-12-28', '2023-12-27', '2023-12-26', '2023-12-25', '2023-12-22', '2023-12-21', '2023-12-20', '2023-12-19', '2023-12-18', '2023-12-15', '2023-12-14', '2023-12-13', '2023-12-12', '2023-12-11', '2023-12-8', '2023-12-7', '2023-12-6', '2023-12-5', '2023-12-4', '2023-12-1', '2023-11-30', '2023-11-29', '2023-11-28', '2023-11-27', '2023-11-23', '2023-11-20', '2023-11-17', '2023-11-16', '2023-11-15', '2023-11-14', '2023-11-13', '2023-11-10', '2023-11-9', '2023-11-8', '2023-11-7', '2023-11-6', '2023-11-3', '2023-11-2', '2023-11-1', '2023-10-31', '2023-10-30', '2023-10-2

In [ ]:
date_code_dict

In [57]:
[input_fliter_txt_num['日期'][i].replace('/','-') for i in range(0,len(input_fliter_txt_num))]

KeyError: 57

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_statistics(vec_dict):
    # 将向量转换为NumPy数组
    vectors = np.array(list(vec_dict.values()))

    # 计算每个维度的均值
    mean = np.mean(vectors, axis=0)

    # 计算每个维度的方差
    variance = np.var(vectors, axis=0)

    # 计算每个维度的最小值
    min_val = np.min(vectors, axis=0)

    # 计算每个维度的最大值
    max_val = np.max(vectors, axis=0)

    # 计算每个维度的第一四分位数、第二四分位数、第三四分位数
    quartiles = np.percentile(vectors, [25, 50, 75], axis=0)
    quartile1, quartile2, quartile3 = quartiles

    # 计算每个维度的偏度
    skewness = skew(vectors, axis=0)

    # 计算每个维度的峰度
    kurt = kurtosis(vectors, axis=0)

    # 将所有统计量合并为一个向量
    stats_vector = np.concatenate((mean, variance, min_val, max_val, quartile1, quartile2, quartile3, skewness, kurt))

    return stats_vector

# 示例用法
vec_dict = {
    'vec1': np.random.rand(100, 3),  # 100个3维向量
    'vec2': np.random.rand(50, 3),   # 50个3维向量
    'vec3': np.random.rand(200, 3)   # 200个3维向量
}

statistics_vector = calculate_statistics(vec_dict)
print(statistics_vector)


In [18]:
len(input_fliter_txt_num)

174